In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi']=400
df = pd.read_csv('cohort_counts.csv')
df.head()

,CohortMonth,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2010-12-01,716.0,246.0,221.0,251.0,245.0,285.0,249.0,236.0,240.0,265.0,254.0,348.0,172.0
1,2011-01-01,332.0,69.0,82.0,81.0,110.0,90.0,82.0,86.0,104.0,102.0,124.0,45.0,NaN
2,2011-02-01,316.0,58.0,57.0,83.0,85.0,74.0,80.0,83.0,86.0,95.0,28.0,NaN,NaN
3,2011-03-01,388.0,63.0,100.0,76.0,83.0,67.0,98.0,85.0,107.0,38.0,NaN,NaN,NaN
4,2011-04-01,255.0,49.0,52.0,49.0,47.0,52.0,56.0,59.0,17.0,NaN,NaN,NaN,NaN


In [132]:
df = df.fillna(0)

In [133]:
df.set_index('CohortMonth', inplace=True)

In [134]:
cohort_sizes = df.iloc[:,0]
cohort_sizes

CohortMonth
2010-12-01    716.0
2011-01-01    332.0
2011-02-01    316.0
2011-03-01    388.0
2011-04-01    255.0
2011-05-01    249.0
2011-06-01    207.0
2011-07-01    173.0
2011-08-01    139.0
2011-09-01    279.0
2011-10-01    318.0
2011-11-01    291.0
2011-12-01     38.0
Name: 1, dtype: float64

In [135]:
retention = df.divide(cohort_sizes, axis=0)
churn = 1 - retention
print(retention)

               1         2         3         4         5         6         7  \
CohortMonth                                                                    
2010-12-01   1.0  0.343575  0.308659  0.350559  0.342179  0.398045  0.347765   
2011-01-01   1.0  0.207831  0.246988  0.243976  0.331325  0.271084  0.246988   
2011-02-01   1.0  0.183544  0.180380  0.262658  0.268987  0.234177  0.253165   
2011-03-01   1.0  0.162371  0.257732  0.195876  0.213918  0.172680  0.252577   
2011-04-01   1.0  0.192157  0.203922  0.192157  0.184314  0.203922  0.219608   
2011-05-01   1.0  0.160643  0.172691  0.144578  0.208835  0.232932  0.244980   
2011-06-01   1.0  0.159420  0.125604  0.198068  0.236715  0.299517  0.091787   
2011-07-01   1.0  0.161850  0.179191  0.219653  0.254335  0.098266  0.000000   
2011-08-01   1.0  0.215827  0.201439  0.251799  0.100719  0.000000  0.000000   
2011-09-01   1.0  0.200717  0.279570  0.121864  0.000000  0.000000  0.000000   
2011-10-01   1.0  0.210692  0.094340  0.

In [136]:
retention_rate = retention.iloc[:,1:].mean().mean()
churn_rate = churn.iloc[:,1:].mean().mean()
print('Retention rate: {:.2f}; Churn rate: {:.2f}'.format(retention_rate, churn_rate))

Retention rate: 0.11; Churn rate: 0.89


In [137]:
online = pd.read_csv('Online12M.csv')

In [138]:
# import datetime as dt
# # def get_month(x): return dt.datetime(x.year, x.month, 1)
online['InvoiceDate'] = pd.to_datetime(online['InvoiceDate'])
# online['InvoiceMonth'] = online['InvoiceDate'].apply(lambda s: dt.datetime(s.year, s.month, 1))

In [139]:
import datetime as dt
online['InvoiceMonth'] = pd.to_datetime(online['InvoiceDate']).dt.to_period('M')

In [140]:
online.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25,2.10,14286,United Kingdom,2011-10
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20,1.45,16360,United Kingdom,2011-11
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14,3.75,13933,United Kingdom,2011-07
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23,2.10,17290,United Kingdom,2011-11
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25,5.95,17663,United Kingdom,2011-05


In [141]:
online.dtypes

Unnamed: 0               int64
InvoiceNo                int64
StockCode               object
Description             object
Quantity                 int64
InvoiceDate     datetime64[ns]
UnitPrice              float64
CustomerID               int64
Country                 object
InvoiceMonth         period[M]
dtype: object

In [143]:
online['TotalSum'] = online['Quantity']*online['UnitPrice']

In [144]:
monthly_revenue = online.groupby(['CustomerID', 'InvoiceMonth'])\
                                 ['TotalSum'].sum().mean()
monthly_revenue = np.mean(monthly_revenue)
lifespan_months = 36
clv_basic = monthly_revenue*lifespan_months
print('Average basiv CLV is {:.1f} USD'.format(clv_basic))
print('Monthly Revenue is: {:.1f} USD'.format(monthly_revenue))

Average basiv CLV is 4726.4 USD
Monthly Revenue is: 131.3 USD


In [145]:
revenue_per_purchase = online.groupby(['InvoiceNo'])['TotalSum'].mean().mean()
freq = online.groupby(['CustomerID', 'InvoiceMonth'])['InvoiceNo'].nunique().mean()
clv_granular = revenue_per_purchase*freq*lifespan_months
print('Average granular CLV is {:.1f} USD'.format(clv_granular))
print('Revenue per purchase: {:.1f} USD'.format(revenue_per_purchase))
print('Frequency per month: {:.2}'.format(freq))

Average granular CLV is 1611.9 USD
Revenue per purchase: 34.6 USD
Frequency per month: 1.3


In [146]:
monthly_revenue = online.groupby(['CustomerID', 'InvoiceMonth'])\
                                 ['TotalSum'].sum().mean()
retention_rate = retention.iloc[:,1:].mean().mean()
churn = 1 - retention_rate
clv_traditonal = monthly_revenue*(retention_rate/churn_rate)
print('Average traditional CLV is {:.1f} USD at {:.1f} % retention_rate'.format(
    clv_traditonal, retention_rate*100))
print('Monthly average revenue is: {:.1f} USD'.format(monthly_revenue))

Average traditional CLV is 17.0 USD at 11.4 % retention_rate
Monthly average revenue is: 131.3 USD


In [147]:
online.groupby(['InvoiceMonth']).size()

InvoiceMonth
2010-12     2205
2011-01     3580
2011-02     3648
2011-03     4764
2011-04     4148
2011-05     5018
2011-06     4669
2011-07     4610
2011-08     4744
2011-09     7189
2011-10     8808
2011-11    11644
2011-12     3149
Freq: M, dtype: int64

In [148]:
online_X = online[online['InvoiceMonth'] != '2011-12-01']
online_X.groupby(['InvoiceMonth']).size()

InvoiceMonth
2010-12     2205
2011-01     3580
2011-02     3648
2011-03     4764
2011-04     4148
2011-05     5018
2011-06     4669
2011-07     4610
2011-08     4744
2011-09     7189
2011-10     8808
2011-11    11644
Freq: M, dtype: int64

In [149]:
online_X.shape

(65027, 11)

In [150]:
online.dtypes

Unnamed: 0               int64
InvoiceNo                int64
StockCode               object
Description             object
Quantity                 int64
InvoiceDate     datetime64[ns]
UnitPrice              float64
CustomerID               int64
Country                 object
InvoiceMonth         period[M]
TotalSum               float64
dtype: object

In [151]:
# online.loc[online['CustomerID'].isin(some_values)]

In [152]:
NOW = dt.datetime(2011,12,1)
NOW

datetime.datetime(2011, 12, 1, 0, 0)

In [153]:
features = online_X.groupby('CustomerID').agg({
                            'InvoiceDate': lambda x: (NOW-x.max()).days,
                            'InvoiceNo': pd.Series.nunique,
                            'TotalSum': np.sum,
                            'Quantity': ['mean', 'sum']
                            }).reset_index()

In [154]:
features.columns = ['CustomerID', 'recency', 'frequency', 
                    'monetary', 'qunatity_avg', 'quantity_total']

In [155]:
features.head()

,CustomerID,recency,frequency,monetary,qunatity_avg,quantity_total
0,12747,14,9,646.40,10.000000,210
1,12748,2,135,6833.36,6.328087,5227
2,12749,14,3,725.85,7.718750,247
3,12820,36,3,202.62,10.307692,134
4,12822,62,2,146.15,9.666667,87


In [156]:
features.shape

(3603, 6)

In [157]:
cust_month_tx = pd.pivot_table(data=online, values='InvoiceNo',
                               index=['CustomerID'], columns=['InvoiceMonth'],
                               aggfunc=pd.Series.nunique, fill_value=0)
cust_month_tx.shape

(3643, 13)

In [159]:
target = ['2011-12-01']
y = cust_month_tx[target]
y.shape

(3643, 1)

In [160]:
cust_month_tx.head()

InvoiceMonth,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12
CustomerID,,,,,,,,,,,,,
12747,1,1,0,1,0,2,1,0,1,0,1,1,1
12748,11,2,4,9,3,17,12,8,9,9,10,41,8
12749,0,0,0,0,0,1,0,0,1,0,0,1,1
12820,0,1,0,0,0,0,0,0,0,1,1,0,1
12822,0,0,0,0,0,0,0,0,0,2,0,0,0


In [161]:
custid = ['CustomerID']
cols = [col for col in features.columns if col not in custid]
X = features[cols]
X.shape

(3603, 5)

In [163]:
compare_online_X = features

In [164]:
compare_online_X.head()

,CustomerID,recency,frequency,monetary,qunatity_avg,quantity_total
0,12747,14,9,646.40,10.000000,210
1,12748,2,135,6833.36,6.328087,5227
2,12749,14,3,725.85,7.718750,247
3,12820,36,3,202.62,10.307692,134
4,12822,62,2,146.15,9.666667,87


In [165]:
compare_online_X.set_index('CustomerID', inplace=True)

In [166]:
x = list(compare_online_X.index.values)
f = list(cust_month_tx.index.values)
missing_indexes = list((set(x) | set(f)) - (set(x) & set(f))) #if you need a list

In [167]:
len(missing_indexes)

40

In [169]:
online.loc[online['CustomerID'].isin(missing_indexes)].head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,TotalSum
358,517830,580094,23359,SET OF 12 T-LIGHTS VINTAGE DOILY,3,2011-12-01,1.95,15904,United Kingdom,2011-12,5.85
760,530167,580819,22757,LARGE RED BABUSHKA NOTEBOOK,24,2011-12-06,0.39,15318,United Kingdom,2011-12,9.36
1385,523964,580526,22078,RIBBON REEL LACE DESIGN,10,2011-12-04,2.10,16535,United Kingdom,2011-12,21.00
1752,529484,580731,22942,CHRISTMAS LIGHTS 10 SANTAS,2,2011-12-05,2.95,17383,United Kingdom,2011-12,5.90
1933,534281,581142,20726,LUNCH BAG WOODLAND,4,2011-12-07,1.65,14204,United Kingdom,2011-12,6.60


In [172]:
y.head()

InvoiceMonth,2011-12
CustomerID,
12747,1
12748,8
12749,1
12820,1
12822,0


In [174]:
y = y.loc[~y.index.isin(missing_indexes)]

In [176]:
y.shape

(3603, 1)

In [177]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [179]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_train_pred = linreg.predict(X_train)
y_test_pred = linreg.predict(X_test)

In [180]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mae_train = np.sqrt(mean_absolute_error(y_train, y_train_pred))

rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mae_test = np.sqrt(mean_absolute_error(y_test, y_test_pred))
# Print the performance metrics
print('RMSE train: {}; RMSE test: {}\nMAE train: {}, MAE test: {}'.format(rmse_train, rmse_test, mae_train, mae_test))

RMSE train: 0.38289494077556424; RMSE test: 0.3752139968718747
MAE train: 0.4676032238545849, MAE test: 0.460717313703017


In [184]:
import statsmodels.api as sm
olsreg = sm.OLS(y_train.values.reshape(-1,1), X_train)
olsreg = olsreg.fit()
print(olsreg.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.362
Model:                            OLS   Adj. R-squared (uncentered):              0.361
Method:                 Least Squares   F-statistic:                              306.1
Date:                Tue, 13 Oct 2020   Prob (F-statistic):                   5.08e-260
Time:                        15:50:26   Log-Likelihood:                         -1240.3
No. Observations:                2702   AIC:                                      2491.
Df Residuals:                    2697   BIC:                                      2520.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------